In [ ]:
import threading
import torch
import time
import dis

In [ ]:
def task():
    print("hello world, from thread", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hello from the main thread,", threading.get_native_id())

### Race condition

- race between adding and printing threads
- who gets further first determines whether the result is correct => we should only run the print thread after running the add thread

In [ ]:
total = 0

def add(count):
    global total
    for i in range(count):
        total += i

t = threading.Thread(target=add, args=[1_000_000])
t.start()
print(total)

In [ ]:
total = 0

def add(count):
    global total
    for i in range(count):
        total += i

t = threading.Thread(target=add, args=[1_000_000])
t.start()
t.join()    # wait for t to finish
print(total)

### Fine-grained locking
- only hold the lock for a small piece of work

In [ ]:
lock = threading.Lock()
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    for i in range(count):
        lock.acquire()
        total += 1
        lock.release()

# inc(1000)
t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

### coarse-grained locking
- don't release lock if you are going to immediately ask for the lock again

In [ ]:
lock = threading.Lock()
total = torch.tensor(0, dtype=torch.int32)

def inc(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

t1 = threading.Thread(target=inc, args=[1_000_000])
t2 = threading.Thread(target=inc, args=[1_000_000])

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()
print(end-start, "seconds")
total

### Bank account example

In [ ]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    lock.acquire()
    success = False
    if bank_accounts[src] >= amount:
        bank_accounts[src] -= amount
        bank_accounts[dst] += amount
        success = True
    print("transferred" if success else "denied")
    lock.release()

In [ ]:
transfer("x", "y", 20)

In [ ]:
transfer("x", "z", 10)

In [ ]:
transfer("w", "x", 50)

What's wrong now?

In [ ]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock:
        #lock.acquire()
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
        # lock.release()    with statement calls automatically, even with exceptions

In [ ]:
transfer("z", "x", 50)